In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import tree
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

In [10]:
# load the data
data_clinical = pd.read_csv("./clinFeatures_UPENN.csv")
data_radiation = pd.read_csv("./radFeatures_UPENN.csv")

# get a small overfiew
print(data_clinical.shape)
print(data_clinical.dtypes)

(611, 10)
SubjectID                      object
Gender                         object
Age_at_scan_years             float64
Survival_from_surgery_days     object
IDH1                           object
MGMT                           object
KPS                            object
GTR_over90percent              object
Time_since_baseline_preop       int64
PsP_TP_score                  float64
dtype: object


In [11]:
data_clinical.head()

,SubjectID,Gender,Age_at_scan_years,Survival_from_surgery_days,IDH1,MGMT,KPS,GTR_over90percent,Time_since_baseline_preop,PsP_TP_score
0,UPENN-GBM-00001_11,F,52.16,Not Available,Wildtype,Not Available,Not Available,Y,0,NaN
1,UPENN-GBM-00002_11,F,61.30,291,Wildtype,Not Available,Not Available,Y,0,NaN
2,UPENN-GBM-00003_11,M,42.82,Not Available,Wildtype,Not Available,Not Available,Y,0,NaN
3,UPENN-GBM-00004_11,M,33.43,Not Available,NOS/NEC,Not Available,Not Available,Y,0,NaN
4,UPENN-GBM-00005_11,M,53.33,Not Available,Wildtype,Not Available,Not Available,Y,0,NaN


In [12]:
print(data_radiation.shape)
print(data_radiation.dtypes)

(611, 4753)
SubjectID                                     object
FLAIR_ED_Intensity_CoefficientOfVariation    float64
FLAIR_ED_Intensity_Energy                      int64
FLAIR_ED_Intensity_InterQuartileRange          int64
FLAIR_ED_Intensity_Kurtosis                  float64
                                              ...   
DSC_PH_ED_NGTDM_Coarsness                    float64
DSC_PH_ED_NGTDM_Complexity                   float64
DSC_PH_ED_NGTDM_Contrast                     float64
DSC_PH_ED_NGTDM_Strength                     float64
DSC_PH_ED_LBP_Radius-1_Bins-16_LBP           float64
Length: 4753, dtype: object


In [13]:
data_radiation.head()

,SubjectID,FLAIR_ED_Intensity_CoefficientOfVariation,FLAIR_ED_Intensity_Energy,FLAIR_ED_Intensity_InterQuartileRange,FLAIR_ED_Intensity_Kurtosis,FLAIR_ED_Intensity_Maximum,FLAIR_ED_Intensity_Mean,FLAIR_ED_Intensity_MeanAbsoluteDeviation,FLAIR_ED_Intensity_Median,FLAIR_ED_Intensity_MedianAbsoluteDeviation,...,DSC_PH_ED_GLSZM_Bins-16_Radius-1_ZoneSizeMean,DSC_PH_ED_GLSZM_Bins-16_Radius-1_ZoneSizeNonUniformity,DSC_PH_ED_GLSZM_Bins-16_Radius-1_ZoneSizeNoneUniformityNormalized,DSC_PH_ED_GLSZM_Bins-16_Radius-1_ZoneSizeVariance,DSC_PH_ED_NGTDM_Busyness,DSC_PH_ED_NGTDM_Coarsness,DSC_PH_ED_NGTDM_Complexity,DSC_PH_ED_NGTDM_Contrast,DSC_PH_ED_NGTDM_Strength,DSC_PH_ED_LBP_Radius-1_Bins-16_LBP
0,UPENN-GBM-00001_11,0.132367,496350786,28,2.614272,223,150.365238,-7.726173e-13,150,0.365238,...,59.930556,59.666667,0.165741,1.186807e+05,159.674492,0.000100,544.462199,0.334864,0.012338,1.997448e+05
1,UPENN-GBM-00002_11,0.183761,5059094750,50,2.191867,255,177.564146,-7.618750e-12,178,-0.435854,...,98.927342,273.911408,0.174577,1.499080e+06,1015.313449,0.000013,545.886998,0.197933,0.002102,1.245890e+06
2,UPENN-GBM-00003_11,0.178541,1285339782,48,2.011637,236,160.387047,-2.585516e-12,164,-3.612953,...,62.805447,153.547341,0.199153,6.902887e+05,446.205555,0.000032,528.910262,0.104093,0.009445,4.014972e+05
3,UPENN-GBM-00004_11,0.178110,604801952,36,2.666624,221,133.854148,7.141096e-13,135,-1.145852,...,46.081690,137.661972,0.193890,7.784522e+04,257.023450,0.000070,527.780251,0.632939,0.006835,2.923168e+05
4,UPENN-GBM-00005_11,0.161688,957903597,29,2.177048,171,117.705824,-8.040465e-13,116,1.705824,...,123.629358,103.957798,0.190748,5.225992e+05,399.581870,0.000024,690.501983,0.801415,0.002235,5.038194e+05


In [19]:
X = data_radiation.iloc
y = data_radiation.iloc

In [20]:
# do a feature selection because 4753 features are way too much
# use Logistig regression
clf = LogisticRegression()
filtered = clf.fit(data_radiation, y)

ValueError: could not convert string to float: 'UPENN-GBM-00001_11'

In [ ]:
# split the data in a train and a test dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

# standardize the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [ ]:
# prediction on the test
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy on training set: ", accuracy_score(y_train, model.predict(X_train)))
print("Accuracy on test set", accuracy_score(y_test, y_pred))